## Trend trading at 1:05PMM

In [45]:
import warnings
warnings.filterwarnings('ignore')

import time
from datetime import date, datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pandas_ta as ta
from ps_candlestick import get_type_candlestick
from ps_candlestick import has_reversal_pattern

In [46]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [47]:
data = dataset.copy()
data["ema_line"] = ta.ema(data["Close"], length=20)
data['above_ma'] = data.apply(lambda r: 1 if r['Close'] > r['ema_line'] else 0, axis=1)
data['below_ma'] = data.apply(lambda r: 1 if r['Close'] < r['ema_line'] else 0, axis=1)
data['total_above_ma'] = data['above_ma'].rolling(150).sum()
data['total_below_ma'] = data['below_ma'].rolling(150).sum()
data['trend'] = data.apply(lambda r: 'switch' if r['total_above_ma'] == r['total_below_ma'] else ('up' if r['total_above_ma'] > r['total_below_ma'] else 'down'), axis=1)

## Strategy

In [48]:
def trade_simulation(prepared_data):
    _trades = pd.DataFrame(columns=("EntryTime", "EntryPrice", "ExitTime", "ExitPrice", "Type", "Profit"))
    has_open_deal = False
    type_open_deal = ''
    # Stoploss at x0 pips
    sl_step = 2
    # Takeprofit at x0 pips(R/R = 1/3)
    tp_step = 6
    for i, row in prepared_data.iterrows():
        if 100*row.name.hour + row.name.minute < 1420:
            if has_open_deal:
                if type_open_deal == 'short':
                    # Stoploss
                    if row['High'] > entry_price + sl_step:
                        profit = -sl_step
                        exit_price = entry_price + sl_step
                        exit_time = row.name
                        _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Short", profit]
                        has_open_deal = False
                        type_open_deal = ''
                    else:
                        # Takeprofit
                        if row['Low'] < entry_price - tp_step:
                            profit = tp_step
                            exit_price = entry_price - tp_step
                            exit_time = row.name
                            _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Short", profit]
                            has_open_deal = False
                            type_open_deal = ''
                elif type_open_deal == 'long':
                    # Stoploss
                    if row['Low'] < entry_price - sl_step:
                        profit = -sl_step
                        exit_price = entry_price - sl_step
                        exit_time = row.name
                        _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Long", profit]
                        has_open_deal = False
                        type_open_deal = ''
                    else:
                        # Takeprofit
                        if row['High'] > entry_price + tp_step:
                            profit = tp_step
                            exit_price = entry_price + tp_step
                            exit_time = row.name
                            _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Long", profit]
                            has_open_deal = False
                            type_open_deal = ''
                        #
            if not has_open_deal:
                if row['signal'] == 'short':
                    # Open short deal
                    entry_price = (row['Close'] + row['Open'])/2
                    entry_time = row.name
                    has_open_deal = True
                    type_open_deal = 'short'
                elif row['signal'] == 'long':
                    # Open short deal
                    entry_price = (row['Close'] + row['Open'])/2
                    entry_time = row.name
                    has_open_deal = True
                    type_open_deal = 'long'
        else:
            if 100*row.name.hour + row.name.minute == 1425:
                if has_open_deal:
                    # close open deal at 2:25PM (dataframe)
                    if type_open_deal == 'short':
                        profit = entry_price - row['Close']
                        if profit < -sl_step:
                            profit = -sl_step
                        exit_price = row['Close']
                        exit_time = row.name
                        has_open_deal = False
                        type_open_deal = ''
                        _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Short", profit]
                    elif type_open_deal == 'long':
                        profit = row['Close'] - entry_price
                        if profit < -sl_step:
                            profit = -sl_step
                        exit_price = row['Close']
                        exit_time = row.name
                        has_open_deal = False
                        type_open_deal = ''
                        _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Long", profit]
    return _trades

In [49]:
def cal_signal_v1(row):
    signal = ''
    if 100 * row.name.hour + row.name.minute == 1300:
        if row['trend'] == 'up' and row['Close'] > row['Open']:
            signal = 'long'
        elif row['trend'] == 'down' and row['Close'] < row['Open']:
            signal = 'short'
    return signal
data['signal'] = data.apply(lambda r: cal_signal_v1(r), axis=1)

In [50]:
trade_result = trade_simulation(data)

In [51]:
trade_result.Profit.sum()

722.7500000000011

In [52]:
trade_result

,EntryTime,EntryPrice,ExitTime,ExitPrice,Type,Profit
0,2018-08-14 13:00:00,953.95,2018-08-14 13:15:00,955.95,Short,-2.00
1,2018-08-15 13:00:00,957.10,2018-08-15 13:50:00,959.10,Short,-2.00
2,2018-08-16 13:00:00,938.60,2018-08-16 13:15:00,936.60,Long,-2.00
3,2018-08-20 13:00:00,950.50,2018-08-20 14:10:00,944.50,Short,6.00
4,2018-08-21 13:00:00,947.75,2018-08-21 14:25:00,954.70,Long,6.95
...,...,...,...,...,...,...
686,2024-06-28 13:00:00,1287.70,2024-06-28 13:45:00,1281.70,Short,6.00
687,2024-07-03 13:00:00,1297.85,2024-07-03 13:30:00,1303.85,Long,6.00
688,2024-07-04 13:00:00,1309.05,2024-07-04 13:25:00,1307.05,Long,-2.00
689,2024-07-05 13:00:00,1309.15,2024-07-05 14:25:00,1316.20,Long,7.05


In [53]:
# win rate
len(trade_result[trade_result.Profit > 0]) / len(trade_result)

0.4283646888567294

In [54]:
def cal_signal_v2(row):
    signal = ''
    if 100 * row.name.hour + row.name.minute == 1300:
        if row['trend'] == 'up':
            signal = 'long'
        elif row['trend'] == 'down':
            signal = 'short'
    return signal
data['signal'] = data.apply(lambda r: cal_signal_v2(r), axis=1)
trade_result = trade_simulation(data)

In [55]:
display_data = {
    'trades' : len(trade_result),
    'profit' : trade_result.Profit.sum(),
    'win rate' : len(trade_result[trade_result.Profit > 0]) / len(trade_result)
}
display_data

{'trades': 1454, 'profit': 298.50000000000114, 'win rate': 0.31636863823933975}

In [56]:
def cal_signal_v3(row):
    signal = ''
    if 100 * row.name.hour + row.name.minute == 1300:
        if row['trend'] == 'up' and row['cs'] == 'green':
            signal = 'long'
        elif row['trend'] == 'down' and row['cs'] == 'red':
            signal = 'short'
    if signal == '':
        if 100 * row.name.hour + row.name.minute == 1305:
            if row['trend'] == 'up' and row['cs'] == 'green':
                signal = 'long'
            elif row['trend'] == 'down' and row['cs'] == 'red':
                signal = 'short'
    return signal
data['cs'] = data.apply(lambda r: 'green' if r['Close'] >= r['Open'] else 'red', axis=1)
# data['cs_shift'] = data['cs'].shift(1)
data['signal'] = data.apply(lambda r: cal_signal_v3(r), axis=1)
trade_result = trade_simulation(data)

In [60]:
total_trades = len(trade_result)
profit = trade_result.Profit.sum()
trading_fee = total_trades * 0.3
win_rate = len(trade_result[trade_result.Profit > 0]) / total_trades
display_data = {
    'trades' : total_trades,
    'profit' : profit,
    'win rate' : win_rate,
    'trading_fee' : trading_fee,
    'profit_after_fee' : profit - trading_fee
}
display_data

{'trades': 1095,
 'profit': 990.4500000000021,
 'win rate': 0.41278538812785387,
 'trading_fee': 328.5,
 'profit_after_fee': 661.9500000000021}

In [58]:
trade_result

,EntryTime,EntryPrice,ExitTime,ExitPrice,Type,Profit
0,2018-08-14 13:00:00,953.95,2018-08-14 13:15:00,955.95,Short,-2.00
1,2018-08-15 13:00:00,957.10,2018-08-15 13:50:00,959.10,Short,-2.00
2,2018-08-16 13:00:00,938.60,2018-08-16 13:15:00,936.60,Long,-2.00
3,2018-08-20 13:00:00,950.50,2018-08-20 14:10:00,944.50,Short,6.00
4,2018-08-21 13:00:00,947.75,2018-08-21 14:25:00,954.70,Long,6.95
...,...,...,...,...,...,...
1090,2024-07-03 13:00:00,1297.85,2024-07-03 13:30:00,1303.85,Long,6.00
1091,2024-07-04 13:00:00,1309.05,2024-07-04 13:25:00,1307.05,Long,-2.00
1092,2024-07-05 13:00:00,1309.15,2024-07-05 14:25:00,1316.20,Long,7.05
1093,2024-07-08 13:05:00,1307.85,2024-07-08 14:25:00,1312.00,Long,4.15
